In [6]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
from time import sleep

list = []
#IMDB - есть страница, на которой находится топ фильмов. На каждой странице по 50 шт, поэтому нам нужен массив list 
#https://www.imdb.com/search/title/?title_type=feature&genres=action&explore=genres&ref_=adv_prv
for i in range(1,1000,50):
    list.append(i)

In [62]:
def parcing(a):
    # функция принимает на вход a - массив чисел. на выходе d - словарь данных
    #заранее определяем наш объект словаря, в котором в списках, последовательно будет лежать инфа о фильмах
    d = { 'name':[], 'year':[],'certificate':[],'runtime':[],'genre':[],'imdb_rating':[],'metascore':[],
        'descr':[],'director':[],'link_to_film':[],'release_date':[],'countries_of_origin':[],
        'filming_locations':[],'production_companies':[],'also_known_as':[]}
    
    for i in a:
        
        url = 'https://www.imdb.com/search/title/?title_type=feature&genres=action&start=' + str(i) + '&explore=genres&ref_=adv_nxt'
        page = requests.get(url)
        soup = BeautifulSoup(page.text)
        
        for j in soup.find_all('div', {'class':'lister-item mode-advanced'}):
            
            #j -один фильм на странице. Собираем название, ссылку из названия на фильм, год выхода фильма
            
            name = j.find('div', {'class':'lister-item-content'}).find('h3', {'class':'lister-item-header'}).a.text
            d['name'].append(name)
            link = j.find('div', {'class':'lister-item-content'}).find('h3', {'class':'lister-item-header'}).a.get('href')
            year = j.find('div', {'class':'lister-item-content'}).find('h3', {'class':'lister-item-header'}).find('span', {'class':'lister-item-year text-muted unbold'}).text
            d['year'].append(year)
            
            print(name)
            
            #Собираем ин-фу из первой строчки - возраст, длительность фильма, жанры
            #для этого создаем дополнительный словарь, в котором инициализируем эти значения как none
            dictionary = {'certificate':'none', 'runtime':'none','genre':'none'}
            
            for k in j.find('p',{'class':'text-muted'}).find_all('span'):
                dictionary[k.get('class')[0]] = k.text
            
            del dictionary['ghost']#в словарь попали вертикальные сепараторы, удаляем их
            
            d['certificate'].append(dictionary['certificate'])
            
            d['runtime'].append(dictionary['runtime'])
                    
            d['genre'].append(dictionary['genre'].strip())

                
            #рейтинг imdb фильма
            
            if(j.find('div', {'class':'inline-block ratings-imdb-rating'})):
                imdb_rating = j.find('div', {'class':'inline-block ratings-imdb-rating'}).strong.text
                d['imdb_rating'].append(imdb_rating)
            else:
                d['imdb_rating'].append('none')
             
            #рейтинг метакритики фильма
                
            if(j.find('div', {'class':'inline-block ratings-metascore'})):
                metascore = j.find('div', {'class':'inline-block ratings-metascore'}).span.text
                d['metascore'].append(metascore)
            else:
                 d['metascore'].append('none')
            
            #краткое описание фильма
            
            descr = j.find_all('p', {'class':'text-muted'})[-1].text.strip()
            d['descr'].append(descr)
            
            #режиссер фильма
            
            director = j.find_all('p')[2].find('a').text
            d['director'].append(director)
        
            
            #идем по ссылке на страницу каждого фильма
            #https://www.imdb.com/title/tt6079772/?ref_=adv_li_tt - пример такой страницы
            #будем брать инфу из большой секции Details
            
            link2film='https://www.imdb.com' + link + '?ref_=adv_li_tt'
            d['link_to_film'].append(link2film)
            page2film = requests.get(link2film)
            soup2film = BeautifulSoup(page2film.text)
            
            #release_date of a film
            
            temp = soup2film.find('div', {'data-testid':'title-details-section'})
            release_date = temp.find('li', {'data-testid':'title-details-releasedate'})
            
            if(release_date):
                release_date_ = release_date.find('li', {'role':'presentation'}).a.text
                d['release_date'].append(release_date_)
            else:
                d['release_date'].append('none')
            
            #countries_of_origin of a film
            
            countries_of_origin = [ ]
            temp2 = soup2film.find('div', {'data-testid':'title-details-section'}).find('li', {'data-testid':'title-details-origin'})
            
            if(temp2):
                for li in temp2.find_all('li'):
                    countries_of_origin.append(li.a.text)
            else:
                countries_of_origin.append('none')
                
            d['countries_of_origin'].append(countries_of_origin)
               
            #filming_locations of a film
            
            filming_locations = [ ]  
            temp3 = soup2film.find('div', {'data-testid':'title-details-section'}).find('li', {'data-testid':'title-details-filminglocations'})
            
            if(temp3):
                for li in temp3.find_all('li'):
                    filming_locations.append(li.a.text)
            else:
                filming_locations.append('none') 
                
            d['filming_locations'].append(filming_locations)
            
            #production companies of a film
            
            production_companies = [ ]  
            temp4 = soup2film.find('div', {'data-testid':'title-details-section'}).find('li', {'data-testid':'title-details-companies'})
            
            if(temp4):
                for li in temp4.find_all('li'):
                    production_companies.append(li.a.text)
            else:
                production_companies.append('none') 
                
            d['production_companies'].append(production_companies)
            
            #film Also known as
            
            also_known_as = [ ]  
            temp5 = soup2film.find('div', {'data-testid':'title-details-section'}).find('li', {'data-testid':'title-details-akas'})
            
            if(temp5):
                for li in temp5.find_all('li'):
                    also_known_as.append(li.span.text)
            else:
                also_known_as.append('none') 
                
            d['also_known_as'].append(also_known_as)
            
            
    return d    

In [63]:
func_result = parcing(list)

Дюна
Дюна
Вечные
Не время умирать
Последняя дуэль
Базз Лайтер
Клыки ночи
Армия воров
Веном 2
Главный герой
Бэтмен
Красное уведомление
Анчартед: На картах не значится
Человек-паук: Нет пути домой
Чёрная Вдова
Шан-Чи и легенда десяти колец
Dune: Part Two
Отряд самоубийц: Миссия навылет
Поездка
Бегущий по лезвию 2049
Веном
Преступный квест
Скорая помощь
Казино Рояль
Охотники за привидениями
Обитель зла: Раккун Сити
007: Спектр
Армия мертвецов
Несправедливость: Боги среди нас
Хороший, плохой, коп
Ворон
Planet Dune
Мстители: Финал
Форсаж 9
007: Координаты «Скайфолл»
Меч короля Артура
Довод
Гладиатор
Матрица: Воскрешение
Темный рыцарь
G.I. Joe: Бросок кобры. Снейк Айз
Бесконечность
Гнев человеческий
Никто
Не дыши 2
Убийца
Лига справедливости Зака Снайдера
Матрица
Every Last One of Them
Квант милосердия
Кодекс киллера
Телохранитель жены киллера
Начало
Властелин колец: Братство кольца
Охотники за привидениями
Круиз по джунглям
Кейт
Флэш
Поезд в Пусан
Моя геройская академия. Фильм 3: Миссия мир

In [66]:
import pandas as pd
df = pd.DataFrame(data = func_result)

In [67]:
df #проверить можно здесь: https://www.imdb.com/search/title/?title_type=feature&genres=action&explore=genres&ref_=adv_prv

,name,year,certificate,runtime,genre,imdb_rating,metascore,descr,director,link_to_film,release_date,countries_of_origin,filming_locations,production_companies,also_known_as
0,Дюна,(2021),12+,155 min,"Action, Adventure, Drama",8.2,74,Feature adaptation of Frank Herbert's science ...,Denis Villeneuve,https://www.imdb.com/title/tt1160419/?ref_=adv...,"September 16, 2021 (Russia)","[United States, Canada]","[Wadi Rum, Jordan]","[Warner Bros., Legendary Entertainment, Villen...",[Dune]
1,Дюна,(1984),12+,137 min,"Action, Adventure, Sci-Fi",6.4,41,A Duke's son leads desert warriors against the...,David Lynch,https://www.imdb.com/title/tt0087182/?ref_=adv...,"December 14, 1984 (United States)","[United States, Mexico]","[Estudios Churubusco Azteca, Mexico City, Dist...","[Dino De Laurentiis Company, Estudios Churubus...",[Dune]
2,Вечные,(2021),18+,157 min,"Action, Adventure, Drama",6.9,53,"The saga of the Eternals, a race of immortal b...",Chloé Zhao,https://www.imdb.com/title/tt9032400/?ref_=adv...,"November 8, 2021 (Russia)","[United Kingdom, United States]","[Fuerteventura, Canary Islands, Spain]","[Marvel Studios, TSG Entertainment, TSG Entert...",[Eternals]
3,Не время умирать,(2021),12+,163 min,"Action, Adventure, Thriller",7.5,68,James Bond has left active service. His peace ...,Cary Joji Fukunaga,https://www.imdb.com/title/tt2382320/?ref_=adv...,"October 7, 2021 (Russia)","[United Kingdom, United States]","[Langvann, Oslo, Norway]","[Metro-Goldwyn-Mayer (MGM), Universal Pictures...",[No Time to Die]
4,Последняя дуэль,(2021),18+,152 min,"Action, Drama, History",7.7,67,King Charles VI declares that Knight Jean de C...,Ridley Scott,https://www.imdb.com/title/tt4244994/?ref_=adv...,"November 18, 2021 (Russia)","[United States, United Kingdom]","[Cahir Castle Tipperary, Ireland]","[20th Century Studios, Pearl Street Films, Sco...",[The Last Duel]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,Хаос,(II) (2005),R,106 min,"Action, Crime, Drama",6.5,none,"Two cops, a rookie and a grizzled vet, pursue ...",Tony Giglio,https://www.imdb.com/title/tt0402910/?ref_=adv...,"June 8, 2006 (Russia)","[Canada, United States, United Kingdom]","[City Centre Motor Hotel - 2111 Main St, Vanco...","[Mobius International, Chaotic Productions, Ch...",[Chaos]
996,Перестрелка,(2016),R,91 min,"Action, Comedy, Crime",6.4,63,"Set in Boston in 1978, a meeting in a deserted...",Ben Wheatley,https://www.imdb.com/title/tt4158096/?ref_=adv...,"April 27, 2017 (Russia)",[United Kingdom],"[Brighton, East Sussex, England, UK]","[Film4, Protagonist Pictures, Rook Films]",[Free Fire]
997,Мобильник,(I) (2016),R,98 min,"Action, Adventure, Horror",4.4,38,When a mysterious cell phone signal causes apo...,Tod Williams,https://www.imdb.com/title/tt0775440/?ref_=adv...,"July 7, 2016 (Russia)",[United States],"[Atlanta, Georgia, USA]","[The Genre Co., Benaroya Pictures, 120dB Films]",[Cell]
998,Дэдпул 3,,none,none,"Action, Comedy, Sci-Fi",none,none,The film is rumored to involve another popular...,Ryan Reynolds,https://www.imdb.com/title/tt6263850/?ref_=adv...,none,[United States],"[Los Angeles, California, USA]",[Marvel Studios],[Deadpool 3]
